## Evaluation notebook

### 1. Install / Import

In [ ]:
import json
from pdf2image import convert_from_path
from common_utils import embed_texts, embed_images, encode_image_to_base64, search_index, retrieve_context, call_gpt_4, extract_figures_from_pdf
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from RAGasClass import RAG
from dotenv import load_dotenv
from config import OPENAI_API_KEY
from PIL import Image
load_dotenv()
from PyPDF2 import PdfReader

### 2. EXTRACT IMAGES

In [ ]:
file ="../knowledge/subset_monetary_policy_report.pdf"

import cv2
import numpy as np
from PIL import Image
import os
import fitz  # PyMuPDF

def extract_images_from_pdf(pdf_path, output_folder="extracted_data", padding=300, xpadding = 300):
    os.makedirs(output_folder, exist_ok=True)  # Create output folder
    image_paths = []
    
    # Open the PDF document
    doc = fitz.open(pdf_path)
    
    # Loop through all pages in the PDF
    for page_index in range(len(doc)):
        page = doc.load_page(page_index)  # Load the page
        
        # Rasterize the page to an image
        pix = page.get_pixmap(dpi=300)  # Convert to image with high DPI
        full_image_path = os.path.join(output_folder, f"full_page_{page_index + 1}.png")
        pix.save(full_image_path)
        
        # Convert the image to OpenCV format (numpy array)
        full_image = cv2.imread(full_image_path)
        
        # Convert to grayscale
        gray_image = cv2.cvtColor(full_image, cv2.COLOR_BGR2GRAY)
        
        # Threshold to create a binary image (to highlight potential image areas)
        _, thresh = cv2.threshold(gray_image, 240, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours (regions that are "boxes" in the image)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Loop through contours and crop the image regions
        img_index = 0
        for contour in contours:
            # Get the bounding box of each contour (x, y, width, height)
            x, y, w, h = cv2.boundingRect(contour)
            
            # Ignore small areas (you can adjust the threshold for min area size)
            if w > 50 and h > 50:
                # Add padding to the bounding box
                x_padded = max(x - padding, 0)  # Ensure x doesn't go below 0
                y_padded = max(y - padding, 0)  # Ensure y doesn't go below 0
                w_padded = min(w + 2 * xpadding, full_image.shape[1] - x_padded)  # Ensure width doesn't exceed image
                h_padded = min(h + 2 * padding, full_image.shape[0] - y_padded)  # Ensure height doesn't exceed image
                
                # Crop the image with padding
                cropped_image = full_image[y_padded:y_padded + h_padded, x_padded:x_padded + w_padded]
                
                # Convert cropped image to PIL format to save it as PNG
                pil_image = Image.fromarray(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                img_filename = f"page_{page_index + 1}_image_{img_index + 1}.png"
                img_path = os.path.join(output_folder, img_filename)
                pil_image.save(img_path, "PNG")
                
                image_paths.append({"image_path": img_path})
                img_index += 1
    
    doc.close()
    return image_paths

# image_paths = extract_images_from_pdf(file)


### 3. EXTRACT TEXT

In [ ]:
def extract_text_from_pdf(pdf_path):
    text_data = []
    # Extract text
    reader = PdfReader(file)
    num_pages = len(reader.pages)

    for page_i in range(num_pages):
        page = reader.pages[page_i]
        page_text = page.extract_text()
        
        if page_text and page_text.strip():
            text_data.append({
                "text": page_text.strip(),
                "page_number": page_i + 1
            })
    return text_data

#text_data = extract_text_from_pdf(file)
#print(text_data)



### 4. Generate questions

In [ ]:
def generate_qa_for_pdf(pdf_path):
    output_json = "QA_" + os.path.basename(pdf_path).replace('.pdf', '.json')

    # Extract images and text from PDF
    image_data = extract_images_from_pdf(pdf_path)
    text_data = extract_text_from_pdf(pdf_path)

    sample_queries = []
    expected_responses = []
    qa_data = []

    # Iterate over text data to generate questions
    for text_info in text_data:
        page_number = text_info["page_number"]
        page_text = text_info["text"]

        user_prompt = json.dumps([
            {"type": "text", "text": (
                "Your task is to formulate a question from the given context while following these rules:\n"
                "1. The question must be answerable using the provided context.\n"
                "2. It should be based on non-trivial information.\n"
                "3. The answer must not contain any links.\n"
                "4. The question should be of moderate difficulty.\n"
                "5. Avoid phrases like 'provided context'.\n"
                "6. The response must be in valid JSON format as follows:\n"
                "{'question': 'Generated question here', 'answer': 'Generated answer here'}"
            )},
            {"type": "text", "text": page_text}
        ])
        
        response_text = call_gpt_4(user_prompt)
        #print(f"Raw API response for text (page {page_number}):", response_text)

        try:
            cleaned_response_text = response_text.strip("```json").strip("```")  # Remove surrounding backticks
            response_data = json.loads(cleaned_response_text)
            question = response_data.get("question")
            answer = response_data.get("answer")

            if question and answer:
                sample_queries.append(question)
                expected_responses.append(answer)
                qa_data.append({"page_number": page_number, "question": question, "answer": answer})
            else:
                print(f"Warning: Missing Q&A data for text on page {page_number}")

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response for text on page {page_number}: {e}")
            print("Response text:", response_text)
            continue

    # Iterate over image data to generate questions
    for image_info in image_data:
        page_number = image_info["image_path"].split("_")[2]  # Extract page number from image path
        image_path = image_info["image_path"]

        # Convert the image to base64 for processing
        image = Image.open(image_path)
        base64_str = encode_image_to_base64(image)

        user_prompt = json.dumps([
            {"type": "text", "text": (
                "Your task is to formulate a question from the given image while following these rules:\n"
                "1. The question must be answerable using the provided image.\n"
                "2. It should be based on non-trivial information.\n"
                "3. The answer must not contain any links.\n"
                "4. The question should be of moderate difficulty.\n"
                "5. Avoid phrases like 'provided image'.\n"
                "6. The response must be in valid JSON format as follows:\n"
                "{'question': 'Generated question here', 'answer': 'Generated answer here'}"
            )},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_str}"}}
        ])

        response_text = call_gpt_4(user_prompt)
        #print(f"Raw API response for image (page {page_number}):", response_text)

        try:
            cleaned_response_text = response_text.strip("```json").strip("```")  # Remove surrounding backticks
            response_data = json.loads(cleaned_response_text)
            question = response_data.get("question")
            answer = response_data.get("answer")

            if question and answer:
                sample_queries.append(question)
                expected_responses.append(answer)
                qa_data.append({"page_number": page_number, "question": question, "answer": answer})
            else:
                print(f"Warning: Missing Q&A data for image on page {page_number}")

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response for image on page {page_number}: {e}")
            print("Response text:", response_text)
            continue

    # Output structured data
    output = {
        "sample_queries": sample_queries,
        "expected_responses": expected_responses,
        "qa_data": qa_data
    }

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=4)

    #print("Final Q&A data saved to", output_json)
    return sample_queries, expected_responses, qa_data

PDF_FILE = "../knowledge/subset_monetary_policy_report.pdf"
sample_queries, expected_responses, qa_data = generate_qa_for_pdf(pdf_path=PDF_FILE)

#### 4.1 (Load from file instead)

In [ ]:
import json

file_path = "/Users/olofswedberg/Documents/thesis/masters-thesis/prototype/QA_subset_monetary_policy_report.json"

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

sample_queries = data["sample_queries"]
expected_responses = data["expected_responses"]
qa_data = data["qa_data"]

### 5. Initialize RAG & Create dataset for eval

In [ ]:
rag = RAG(OPENAI_API_KEY)

# Load documents
pdf_path_list = [PDF_FILE]
rag.load_documents(pdf_path_list)

# Generate dataset
dataset = []

for query,reference in zip(sample_queries,expected_responses):

    relevant_docs = rag.get_most_relevant_docs(query)
    response = rag.generate_answer(query, relevant_docs)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":relevant_docs,
            "response":response,
            "reference":reference
        }
    )

# Save the dataset to a JSON file
output_dataset_file = "generated_dataset.json"
with open(output_dataset_file, 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

#### 5.1 (Load the dataset to save time)

In [ ]:
# Code to load the dataset
with open(output_dataset_file, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

### 6. Evaluate using RAGAs test

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from RAGasClass import RAG
from config import OPENAI_API_KEY

llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings()

# Initialize RAG instance
rag = RAG(OPENAI_API_KEY)

sample_queries = [
    "Who introduced the theory of relativity?",
    "Who was the first computer programmer?",
    "What did Isaac Newton contribute to science?",
    "Who won two Nobel Prizes for research on radioactivity?",
    "What is the theory of evolution by natural selection?"
]

expected_responses = [
    "Albert Einstein proposed the theory of relativity, which transformed our understanding of time, space, and gravity.",
    "Ada Lovelace is regarded as the first computer programmer for her work on Charles Babbage's early mechanical computer, the Analytical Engine.",
    "Isaac Newton formulated the laws of motion and universal gravitation, laying the foundation for classical mechanics.",
    "Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity and won two Nobel Prizes.",
    "Charles Darwin introduced the theory of evolution by natural selection in his book 'On the Origin of Species'."
]

sample_docs = [
    "Albert Einstein proposed the theory of relativity, which transformed our understanding of time, space, and gravity.",
    "Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity and won two Nobel Prizes.",
    "Isaac Newton formulated the laws of motion and universal gravitation, laying the foundation for classical mechanics.",
    "Charles Darwin introduced the theory of evolution by natural selection in his book 'On the Origin of Species'.",
    "Ada Lovelace is regarded as the first computer programmer for her work on Charles Babbage's early mechanical computer, the Analytical Engine."
]

# Load documents
# pdf_path_list = ["sample_docs.pdf"]
# rag.load_documents(pdf_path_list)
rag.load_documents(sample_docs)


# Query and retrieve the most relevant document
query = "Who introduced the theory of relativity?"
relevant_docs = rag.get_most_relevant_docs(query)
answer = rag.generate_answer(query, relevant_docs)

print(f"Query: {query}")
print(f"Relevant Document: {relevant_docs}")
print(f"Answer: {answer}")





#### 6.1 Prepare the data for RAGAs

In [ ]:
dataset = []

for query,reference in zip(sample_queries,expected_responses):
    relevant_docs = rag.get_most_relevant_docs(query)
    response = rag.generate_answer(query, relevant_docs)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts": [doc['content'] for doc in  relevant_docs],
            "response":response,
            "reference":reference
        }
    )

for i, entry in enumerate(dataset):
    print(f"Entry {i+1}:")
    print(f"  User Input: {entry['user_input']}")
    print(f"  Retrieved Contexts: {entry['retrieved_contexts']}")
    print(f"  Response: {entry['response']}")
    print(f"  Reference: {entry['reference']}")
    print("-" * 40)


from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

#### 6.2 Evaluate using RAGAs

In [4]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper


evaluator_llm = LangchainLLMWrapper(llm)
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result

Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Evaluating: 100%|██████████| 15/15 [00:30<00:00,  2.01s/it]


{'context_recall': 0.6000, 'faithfulness': 0.4104, 'factual_correctness': 0.6360}